# Data Cleaning Starter Notebook

Use this notebook to load raw ERCOT monthly CSV files and build cleaned tables.

In [ ]:
from pathlib import Path
import pandas as pd

print('pandas:', pd.__version__)

In [ ]:
RAW_ROOT = Path('data/raw/ercot')
PROCESSED_ROOT = Path('data/processed/ercot')
PROCESSED_ROOT.mkdir(parents=True, exist_ok=True)

datasets = sorted([p.name for p in RAW_ROOT.iterdir() if p.is_dir()]) if RAW_ROOT.exists() else []
print('Raw datasets found:', datasets)

In [ ]:
def read_monthly_csv(dataset_id: str, usecols=None) -> pd.DataFrame:
    files = sorted((RAW_ROOT / dataset_id).glob('*/*/*.csv'))
    if not files:
        raise FileNotFoundError(f'No CSV files found for {dataset_id}')
    return pd.concat((pd.read_csv(f, usecols=usecols) for f in files), ignore_index=True)

def parse_hour_ending(series: pd.Series) -> pd.Series:
    s = series.astype(str).str.strip().str.split(':').str[0]
    return pd.to_numeric(s, errors='coerce')

In [ ]:
# Example 1: hourly load (NP6-346-CD)
try:
    load = read_monthly_csv('NP6-346-CD', usecols=['OperDay', 'HourEnding', 'TOTAL'])
    he = parse_hour_ending(load['HourEnding'])
    load['ts_local'] = pd.to_datetime(load['OperDay'], format='%m/%d/%Y', errors='coerce') + pd.to_timedelta(he - 1, unit='h')
    load = load.rename(columns={'TOTAL': 'load_mw'}).drop_duplicates(subset=['ts_local']).sort_values('ts_local')
    print('NP6-346 rows:', len(load), 'range:', load['ts_local'].min(), 'to', load['ts_local'].max())
    display(load.head())
except Exception as exc:
    print('NP6-346 sample skipped:', exc)

In [ ]:
# Example 2: interval price (NP6-905-CD) -> hourly mean
try:
    price = read_monthly_csv('NP6-905-CD', usecols=['DeliveryDate', 'DeliveryHour', 'DeliveryInterval', 'SettlementPointPrice'])
    d = pd.to_datetime(price['DeliveryDate'], format='%m/%d/%Y', errors='coerce')
    dh = pd.to_numeric(price['DeliveryHour'], errors='coerce')
    di = pd.to_numeric(price['DeliveryInterval'], errors='coerce')
    price['ts_15m'] = d + pd.to_timedelta(dh - 1, unit='h') + pd.to_timedelta((di - 1) * 15, unit='m')
    price_hourly = (
        price.groupby(pd.Grouper(key='ts_15m', freq='H'), as_index=False)['SettlementPointPrice']
             .mean()
             .rename(columns={'ts_15m': 'ts_local', 'SettlementPointPrice': 'rt_price_mean'})
             .sort_values('ts_local')
    )
    print('NP6-905 hourly rows:', len(price_hourly), 'range:', price_hourly['ts_local'].min(), 'to', price_hourly['ts_local'].max())
    display(price_hourly.head())
except Exception as exc:
    print('NP6-905 sample skipped:', exc)

In [ ]:
# Optional: write cleaned sample outputs
# Uncomment after validating results
# out_dir = PROCESSED_ROOT / 'examples'
# out_dir.mkdir(parents=True, exist_ok=True)
# load.to_parquet(out_dir / 'np6_346_hourly.parquet', index=False)
# price_hourly.to_parquet(out_dir / 'np6_905_hourly_mean.parquet', index=False)